First define two Variational Autoencoder (VAE) models, `VAE1` and `VAE2`. Both models have an encoder and a decoder, and use the reparameterization trick to sample from the latent space. The difference between the two models is the number of hidden layers in the encoder and decoder.

Next, initialize the model and create a data loader for the dataset. Also initialize the optimizer and define the loss function, which is a combination of reconstruction loss and KL divergence loss.

Finally, train the model for a specified number of epochs, updating the model parameters with each batch of data. The loss for each epoch is printed to monitor the training process.

# Step 3: Build the Model
## Solution 4
* VAE model

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Convert the dataset to a PyTorch DataLoader
data_loader = torch.utils.data.DataLoader(data, batch_size=32, shuffle=True)
latent_dim = 64

class VAE1(nn.Module):
    """Variational Autoencoder with 1 hidden layer in the encoder and decoder."""
    def __init__(self):
        super(VAE1, self).__init__()

        # Encoder
        self.fc1 = nn.Linear(5, 256)
        self.fc2 = nn.Linear(256, latent_dim) # mu
        self.fc3 = nn.Linear(256, latent_dim) # logvar

        # Decoder
        self.fc4 = nn.Linear(latent_dim, 256)
        self.fc5 = nn.Linear(256, 5)

    def encoder(self, x):
        """Encode the inputs to latent space."""
        h1 = F.relu(self.fc1(x))
        return self.fc2(h1), self.fc3(h1)

    def reparameterize(self, mu, logvar):
        """Reparameterization trick to sample from the latent space."""
        std = torch.exp(0.5*logvar) # calculate std
        eps = torch.randn_like(std) # random
        return mu + eps*std

    def decoder(self, z):
        """Decode the latent variables to the original space."""
        h3 = F.relu(self.fc4(z))
        return torch.sigmoid(self.fc5(h3))

    def forward(self, x):
        """Forward pass through the network."""
        mu, logvar = self.encoder(x.view(-1, 5))
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Initialize the model
model = VAE1()



In [ ]:
# Create a data loader
dataset = TensorDataset(torch.tensor(data))
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

class VAE2(nn.Module):
    """Variational Autoencoder with 2 hidden layers in the encoder and decoder."""
    def __init__(self, input_dim=5, latent_dim=latent_dim):
        super(VAE2, self).__init__()

        # Define the encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim * 2)  # output a mean and a log variance
        )

        # Define the decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim),
            nn.Sigmoid()  # output is between 0 and 1
        )

    def reparameterize(self, mu, logvar):
        """Reparameterization trick to sample from the latent space."""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        """Forward pass through the network."""
        mu_logvar = self.encoder(x.view(-1, 5)).view(-1, 2, latent_dim)
        mu = mu_logvar[:, 0, :]
        logvar = mu_logvar[:, 1, :]
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Initialize the model
# model = VAE2()

# Step 4: Train the Model

In [ ]:

# Initialize the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
EPOCHS = 100

def loss_function(recon_x, x, mu, logvar, mse=True):
    """Define the loss function: Reconstruction + KL divergence losses summed over all elements and batch."""
    if mse:
        MSE = F.mse_loss(recon_x, x, reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return MSE + KLD
    else:
        BCE = F.binary_cross_entropy(recon_x, x.view(-1, 5), reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return BCE + KLD

# Train the model
for epoch in range(EPOCHS):
    for batch in data_loader:
        batch = batch.to(model.fc1.weight.dtype)
        model.zero_grad()
        recon_batch, mu, logvar = model(batch)
        loss = loss_function(recon_batch, batch, mu, logvar)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch}, Loss {loss.item()}')